In [1]:
# !pip install -U spacy
# !python3 -m spacy download en_core_web_sm
from pprint import pprint
import string
import numpy as np
import jsonlines
p=string.punctuation

In [2]:
import pandas as pd
import spacy
nlp=spacy.load('en_core_web_lg')
BASEPATH = "../data/"

In [3]:
train = dict()
val = dict()

basepath = "../data/"

with jsonlines.open(basepath+'ade20k-train-combined.jsonl','r') as f:
    for row in f:
        train.update(row)        

with jsonlines.open(basepath+'ade20k-val-combined.jsonl','r') as f:
    for row in f:
        val.update(row)        
len(train.keys()), len(val.keys())



(20210, 2000)

In [4]:
train = pd.DataFrame(train).T.reset_index(level=0)[['index','caption']].rename(columns={'index':'img_id'})
val = pd.DataFrame(val).T.reset_index(level=0)[['index','caption']].rename(columns={'index':'img_id'})

In [5]:
train

,img_id,caption
0,ADE_train_00003661,In this picture I can see the inside view of a...
1,ADE_train_00003722,This is a picture taken inside of a room in th...
2,ADE_train_00013827,"In this image we can see monitors, keyboards, ..."
3,ADE_train_00009835,There is a building at the bottom of this imag...
4,ADE_train_00002932,At the bottom of the image there is a bed with...
...,...,...
20205,ADE_train_00014320,In this picture I can observe some dried leave...
20206,ADE_train_00006757,"In this picture we can see chairs, objects on ..."
20207,ADE_train_00002858,"In this image. In the foreground, I can see a ..."
20208,ADE_train_00003918,"In this image, there are different color pillo..."


In [6]:
import nltk
import collections
from nltk.tokenize import wordpunct_tokenize

In [7]:
sw = list(nlp.Defaults.stop_words)
# len(sw)

from nltk.corpus import stopwords
# nltk.download('stopwords')
sw.extend(stopwords.words("english"))

bad_words = ['', '0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 
             'I', 'a', 'above', 'also', 'an', 'and', 'at', 'back', 'background', 'be', 'below',
             'behind', 'beside', 'bottom', 'center', 'can', 'corner', 'different','few', 'five', 'foreground','f',
             'four', 'front', 'group', 'here', 'image', 'is', 'it', 'in','into','item','items','number',
             'just','kind','left', 'like', 'looks', 'look','many', 'middle', 
             'object', 'objects', 'of', 'one', 'or', 'other', 'on','onto',
             'photo', 'picture', 'placed', 'photograph', 'right', 
             's', 'see', 'seems', 'seen', 'seven', 
             'side', 'six', 'so', 'some', 't', 'taken', 'that', 'the', 'there',
             'them', 'this', 'three', 'they','u',
             'to', 'top', 'two', 'view', 'we', 'which', 'under','zero']

sw.extend(bad_words)

# print(sw)
len(sw)

599

In [8]:
data = []

lines = list(train.caption)

tokens = []
tokens.extend([w.lower() for line in lines for w in wordpunct_tokenize(line)])

len(lines), len(tokens)


(20210, 983863)

In [20]:
#Unigrams

unigrams = collections.Counter(tokens).most_common(5000)
unigrams = [t[0] for t in unigrams]
unigrams = [row.translate(str.maketrans('', '', p)) for row in unigrams]
unigrams = [row for row in unigrams if len(row)>1]
unigrams = list(set(unigrams))
unigrams = [row for row in unigrams if row not in sw]
# data.extend(unigrams)
len(unigrams)

3018

In [154]:
#Bigrams

# bigram_measures = nltk.collocations.BigramAssocMeasures()
# finder = nltk.collocations.BigramCollocationFinder.from_words(tokens)
# finder.apply_freq_filter(5)
# bigram_tuples = finder.nbest(bigram_measures.pmi, 10000)
# bigrams = []
# for bigram_tuple in bigram_tuples:
#     if not(bigram_tuple[0] in sw and bigram_tuple[1] in sw):
#         bigrams.append(' '.join(bigram_tuple))
# bigrams = [row.translate(str.maketrans('', '', p)).strip() for row in bigrams]
# trigrams = [row for row in bigrams if len(row)>0]

# # data.extend(bigrams)
# len(bigrams)

7027

In [22]:
# trigram_measures = nltk.collocations.TrigramAssocMeasures()
# finder = nltk.collocations.TrigramCollocationFinder.from_words(tokens)
# finder.apply_freq_filter(5)
# trigram_tuples = finder.nbest(trigram_measures.pmi, 10000)
# trigrams = []
# for trigram_tuple in trigram_tuples:
#     c = 0
#     for t in trigram_tuple:
#         if t in sw:
#             c+=1
#     if c < 2:
#         trigrams.append(' '.join(trigram_tuple))
# trigrams = [row.translate(str.maketrans('', '', p)).strip() for row in trigrams]
# trigrams = [row for row in trigrams if len(row)>0]

# len(trigrams)

In [300]:
import requests

coco_file = requests.get("https://the-eye.eu/public/AI/models/antarctic-captions/postcache.txt")

In [23]:
# c = []
# for line in coco_file.text.splitlines():
#     c.append(line)
    
# len(c)

# coco = []
# for row in c:
#     l = row.split()
#     if len(l) == 1:
#         if row not in sw: 
#             coco.append(row)
# #     if len(l) == 2:
# #         if not(l[0] in sw and l[1] in sw):
# #             coco.append(row)
            
# coco = [row.translate(str.maketrans('', '', p)).strip() for row in coco]
# coco = [row for row in coco if len(row)>0]

# len(coco)

In [24]:
data = []
data.extend(unigrams)
# data.extend(bigrams)
# data.extend(trigrams)
# data.extend(coco)
grams = list(set(data))

len(data)

3018

In [25]:
%cd ../antarctic-captions/

/project/dataset-exploration/clip-finetune-ade20k/antarctic-captions


In [26]:
import os
from CLIP import clip
import torch
from tqdm import tqdm

CHECKPOINT = "../training-scripts/open_clip/logs/low-lr-low-wd-rn50/checkpoints/epoch_6.pt"

device = "cuda:1" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("RN50",device=device,jit=True) #Must set jit=False for training

In [27]:
checkpoint = torch.load(CHECKPOINT)

# # Use these 3 lines if you use default model setting(not training setting) of the clip. For example, if you set context_length to 100 since your string is very long during training, then assign 100 to checkpoint['model_state_dict']["context_length"] 
checkpoint['state_dict']["input_resolution"] = model.input_resolution #default is 224
checkpoint['state_dict']["context_length"] = model.context_length # default is 77
checkpoint['state_dict']["vocab_size"] = model.vocab_size 

model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [28]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [29]:
def get_text_features(texts):
    all_text_features = []
    for i, x in enumerate(batch(texts, 3000)):
        print("Batch: ", i)
        with torch.no_grad():
            text_inputs = torch.cat([clip.tokenize(text) for text in x]).to(device)
            text_features = model.encode_text(text_inputs)
            all_text_features.extend(text_features.cpu().numpy())

    return all_text_features

In [30]:
embeddings = get_text_features(grams)

Batch:  0
Batch:  1


In [32]:
len(embeddings), len(grams)

(3018, 3018)

In [42]:
.shape

(3018, 1024)

In [49]:
d = "/home/users/ragarwal/project/dataset-exploration/clip-finetune-ade20k/outputs/ade20k-clean-index/"
np.save(d+"emb.npy", np.array(embeddings))

with open(d+"grams.txt", 'w') as f:
    for row in grams:
        f.write(row+"\n")


In [35]:
d = dict(zip(grams, embeddings))

In [249]:
from sentence_transformers.util import semantic_search

In [323]:
from scipy.special import softmax
def test_text(t, dd):
    emb = get_text_features([t])[0]
    k = list(dd.keys())
    v = list(dd.values())
    predict = semantic_search(torch.Tensor(emb), torch.Tensor(v), top_k=50)[0]
    
    for i in predict:
        print(k[i['corpus_id']], i['score'])

    return predict

In [349]:
x = test_text("bad guy ", d)

Batch:  0
guy 0.8982160687446594
badly 0.8881394267082214
bad 0.8864423632621765
aboy 0.8847781419754028
hes 0.8762815594673157
daddy 0.8754550218582153
babay 0.8750624060630798
baby 0.8746769428253174
killer 0.8727972507476807
gunman 0.8722986578941345
boul 0.8717570900917053
goody 0.8699916005134583
kill 0.8697551488876343
cool 0.8686541318893433
boss 0.8684195876121521
highly 0.8681061863899231
goodly 0.8675404191017151
cooly 0.8669133186340332
heard 0.8658984303474426
baord 0.8658579587936401
good 0.8655068874359131
yeah 0.8654959797859192
mister 0.865454912185669
thats 0.8652703166007996
killed 0.8650326728820801
told 0.8645067811012268
babe 0.864257276058197
king 0.8641729950904846
girll 0.8634682297706604
mans 0.8628709316253662
hard 0.8627486228942871
sayd 0.8623270392417908
said 0.8620197772979736
beast 0.8617669343948364
style 0.8617085814476013
boddy 0.8610024452209473
reminds 0.8609949350357056
personified 0.8606639504432678
slyly 0.860624372959137
swag 0.8604247570037842
w

In [325]:
x[0]

{'corpus_id': 1608, 'score': 0.8984373211860657}

In [326]:
np.array(list(d.values())).T

array([[-0.10596 ,  0.3735  ,  0.42    , ..., -0.02908 ,  0.4182  ,
        -0.0876  ],
       [-0.09143 ,  0.2312  , -0.215   , ...,  0.429   , -0.1384  ,
         0.04523 ],
       [ 0.0877  ,  0.1052  ,  0.05008 , ..., -0.1053  , -0.3806  ,
        -0.179   ],
       ...,
       [ 0.07495 ,  0.2932  , -0.1362  , ...,  0.11414 ,  0.0569  ,
        -0.4634  ],
       [-0.3123  , -0.006054, -0.2031  , ..., -0.3076  , -0.1565  ,
        -0.1428  ],
       [ 0.2266  ,  0.178   , -0.1002  , ..., -0.0394  ,  0.2021  ,
         0.00836 ]], dtype=float16)

In [350]:
from scipy.special import softmax
def test_img(dd, img_features):
    
    k = list(dd.keys())
    v = list(dd.values())
    predict = semantic_search(torch.Tensor(img_features), torch.Tensor(v), top_k=20)[0]
    
    for i in predict:
        print(k[i['corpus_id']], i['score'])

    return predict

In [328]:
train_img_embedding_pairs = np.load("../outputs/clip-embeddings/finetuned-img-embeddings-train.npy", allow_pickle=True)
train_img_embedding_pairs = train_img_embedding_pairs.item()

val_img_embedding_pairs = np.load("../outputs/clip-embeddings/finetuned-img-embeddings-val.npy", allow_pickle=True)
val_img_embedding_pairs = val_img_embedding_pairs.item()



In [352]:
x = train_img_embedding_pairs['ADE_train_00008485']

In [353]:
test_img(d, x)

spotlights 0.22547604143619537
leds 0.21454814076423645
garage 0.2127702385187149
hid 0.2120674103498459
headlights 0.2069370150566101
headlamps 0.2043430209159851
spotlighting 0.20350639522075653
garages 0.2023019641637802
downlight 0.19860047101974487
sidelights 0.1964808702468872
installed 0.19610333442687988
indicators 0.19545799493789673
lights 0.19388195872306824
led 0.1924658715724945
extinguishers 0.1918090283870697
crusiers 0.19167503714561462
headlight 0.1911206990480423
ligths 0.18996240198612213
lightings 0.18899071216583252
mazda 0.1873607337474823


[{'corpus_id': 1343, 'score': 0.22547604143619537},
 {'corpus_id': 11121, 'score': 0.21454814076423645},
 {'corpus_id': 11160, 'score': 0.2127702385187149},
 {'corpus_id': 19717, 'score': 0.2120674103498459},
 {'corpus_id': 22148, 'score': 0.2069370150566101},
 {'corpus_id': 22126, 'score': 0.2043430209159851},
 {'corpus_id': 6834, 'score': 0.20350639522075653},
 {'corpus_id': 8232, 'score': 0.2023019641637802},
 {'corpus_id': 4487, 'score': 0.19860047101974487},
 {'corpus_id': 18602, 'score': 0.1964808702468872},
 {'corpus_id': 4674, 'score': 0.19610333442687988},
 {'corpus_id': 3515, 'score': 0.19545799493789673},
 {'corpus_id': 3155, 'score': 0.19388195872306824},
 {'corpus_id': 17199, 'score': 0.1924658715724945},
 {'corpus_id': 3340, 'score': 0.1918090283870697},
 {'corpus_id': 7664, 'score': 0.19167503714561462},
 {'corpus_id': 9169, 'score': 0.1911206990480423},
 {'corpus_id': 2011, 'score': 0.18996240198612213},
 {'corpus_id': 11654, 'score': 0.18899071216583252},
 {'corpus_id'

In [333]:
x

array([ 0.03397,  0.04382, -0.03583, ...,  0.0169 ,  0.0589 ,  0.01245],
      dtype=float16)